In [116]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, make_scorer, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [117]:
# Chargement des données
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')



# Sélection des features
numeric_features = ['max_power', 'weight_min', 'weight_max', 'urb_cons', 
                    'exturb_cons', 'overall_cons', 'co', 'hc', 'nox', 'hcnox', 'ptcl']
categorical_features = [ 'brand', 'range', 'grbx_type_ratios', 'car_class', 'fuel_type', 'hybrid']

# Créer une copie pour éviter de modifier les données originales
X_train = train_df.copy()
X_test = test_df.copy()

# Dropping things:
# Model:
X_train.drop('model', axis=1, inplace=True)
X_test.drop('model', axis=1, inplace=True)
# ID:
X_train = X_train.drop('id', axis=1)
X_test = X_test.drop('id', axis=1)


# # Brand:
# X_train = X_train.drop('brand', axis=1)
# X_test = X_test.drop('brand', axis=1)
# # grbx_type_ratios:
# X_train = X_train.drop('grbx_type_ratios', axis=1)
# X_test = X_test.drop('grbx_type_ratios', axis=1)
# # range:
# X_train = X_train.drop('range', axis=1)
# X_test = X_test.drop('range', axis=1)
    

In [118]:


# 1. Gestion des valeurs manquantes et standardisation des variables numériques
for col in numeric_features:
    combined_values = pd.concat([X_train[col], X_test[col]])
    mean_value = combined_values.mean()
    std_value = combined_values.std()
    
    X_train[col] = X_train[col].fillna(mean_value)
    X_test[col] = X_test[col].fillna(mean_value)
    
    X_train[col] = (X_train[col] - mean_value) / std_value
    X_test[col] = (X_test[col] - mean_value) / std_value

#Standardisation de co2 et création de y_train
co2_mean = X_train['co2'].mean()
co2_std = X_train['co2'].std()
X_train['co2'] = (X_train['co2'] - co2_mean) / co2_std
y_train = X_train.pop('co2')










# 2. Encodage des variables catégoriques avec OneHotEncoder

# Initialisation du OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit sur les données d'entraînement uniquement
encoder.fit(X_train[categorical_features])

# Transformation des données d'entraînement et de test
encoded_train = encoder.transform(X_train[categorical_features])
encoded_test = encoder.transform(X_test[categorical_features])

# Création de DataFrame à partir des données encodées avec les noms des catégories
encoded_feature_names = []
for i, feature in enumerate(categorical_features):
    categories = encoder.categories_[i]
    for category in categories:
        encoded_feature_names.append(f"{feature}_{category}")
    

# Conversion des données encodées en DataFrame
encoded_train_df = pd.DataFrame(encoded_train, columns=encoded_feature_names, index=X_train.index)
encoded_test_df = pd.DataFrame(encoded_test, columns=encoded_feature_names, index=X_test.index)

# Suppression des colonnes catégoriques originales et ajout des colonnes encodées
X_train_numeric = X_train.drop(categorical_features, axis=1)
X_test_numeric = X_test.drop(categorical_features, axis=1)

# Concaténation des variables numériques et catégoriques encodées
X_train = pd.concat([X_train_numeric, encoded_train_df], axis=1)
X_test = pd.concat([X_test_numeric, encoded_test_df], axis=1)

# Vérification du résultat
print(f"Forme des données d'entraînement finales: {X_train.shape}")
print(f"Forme des données de test finales: {X_test.shape}")

print(X_train.info())
train_df.head()


# 2. Encodage des variables catégoriques avec LabelEncoder au lieu de OneHotEncoder

# # Initialisation des LabelEncoders pour chaque feature catégorique
# label_encoders = {}
# for feature in categorical_features:
#     label_encoders[feature] = LabelEncoder()
    
#     # Fit sur les données d'entraînement uniquement
#     X_train[feature] = label_encoders[feature].fit_transform(X_train[feature].fillna('unknown'))
    
#     # Transformation des données de test
#     # Gérer les catégories inconnues dans l'ensemble de test
#     X_test[feature] = X_test[feature].fillna('unknown')
#     # Pour gérer les valeurs dans X_test qui n'existent pas dans X_train
#     for val in np.unique(X_test[feature]):
#         if val not in label_encoders[feature].classes_:
#             # Ajouter cette classe à l'encodeur
#             label_encoders[feature].classes_ = np.append(label_encoders[feature].classes_, val)
    
#     X_test[feature] = label_encoders[feature].transform(X_test[feature])

# # Assurez-vous que toutes les colonnes sont numériques
# X_train = X_train.astype(float)
# X_test = X_test.astype(float)

# # Vérification du résultat
# print(f"Forme des données d'entraînement finales: {X_train.shape}")
# print(f"Forme des données de test finales: {X_test.shape}")

# X_train = X_train.drop('id', axis=1)
# X_test = X_test.drop('id', axis=1)

# print(X_train.info())
# X_train.head()


Forme des données d'entraînement finales: (41257, 102)
Forme des données de test finales: (13753, 102)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41257 entries, 0 to 41256
Columns: 102 entries, max_power to hybrid_oui
dtypes: float64(102)
memory usage: 32.1 MB
None


,id,brand,model,car_class,range,fuel_type,hybrid,max_power,grbx_type_ratios,weight_min,weight_max,urb_cons,exturb_cons,overall_cons,co,hc,nox,hcnox,ptcl,co2
0,0,MERCEDES,COMBI 110 CDI,MINIBUS,MOY-INFER,GO,non,70.0,M 6,1976,2075,9.1,6.4,7.4,0.083,NaN,0.229,0.250,0.001,195
1,1,MERCEDES,VIANO 2.0 CDI,MINIBUS,MOY-SUPER,GO,non,100.0,A 5,2186,2355,10.2,7.0,8.2,0.078,NaN,0.224,0.233,0.001,216
2,2,MERCEDES,SPRINTER COMBI 319 CDI,MINIBUS,MOY-INFER,GO,non,140.0,A 5,2586,2869,12.5,9.0,10.3,0.067,0.014,1.846,NaN,0.002,272
3,3,RENAULT,MEGANE Coupé EnergyTCe (115ch) eco2,COUPE,MOY-INFER,ES,non,85.0,M 6,1280,1280,6.4,4.6,5.3,0.167,0.039,0.039,NaN,0.001,119
4,4,MERCEDES,COMBI 116 CDI,MINIBUS,MOY-INFER,GO,non,120.0,A 5,2356,2450,10.1,6.9,8.1,0.042,NaN,0.190,0.201,0.001,214


In [119]:
# Configurer pandas pour afficher toutes les lignes et colonnes
pd.set_option('display.max_rows', None)      # Afficher toutes les lignes
pd.set_option('display.max_columns', None)   # Afficher toutes les colonnes
pd.set_option('display.width', None)         # Largeur d'affichage automatique
pd.set_option('display.float_format', '{:.3f}'.format)  # Limiter à 3 décimales pour la lisibilité

# Calculer la matrice de corrélation
corr_matrix = (pd.concat([X_train, y_train], axis=1)).corr().abs()

# Visualiser les corrélations avec la variable cible
co2_correlations = corr_matrix['co2'].sort_values(ascending=False)
print(co2_correlations)

# Si vous voulez revenir aux paramètres par défaut après l'affichage
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.float_format')


co2                             1.000
overall_cons                    0.971
exturb_cons                     0.966
urb_cons                        0.907
weight_min                      0.641
car_class_MINIBUS               0.585
weight_max                      0.539
car_class_BERLINE               0.520
grbx_type_ratios_A 5            0.454
hcnox                           0.437
max_power                       0.346
range_INFERIEURE                0.316
grbx_type_ratios_M 5            0.290
range_SUPERIEURE                0.248
brand_MERCEDES                  0.247
nox                             0.243
ptcl                            0.201
grbx_type_ratios_M 6            0.201
car_class_BREAK                 0.198
brand_FIAT                      0.175
hybrid_non                      0.174
hybrid_oui                      0.174
brand_OPEL                      0.174
range_ECONOMIQUE                0.171
car_class_MONOSPACE COMPACT     0.170
hc                              0.164
grbx_type_ra

In [120]:


X_train_nn, X_val, y_train_nn, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

In [121]:

NUM_EPOCHS = 100
BATCH_SIZE = 64
LR = 1e-5
WEIGHT_DECAY = 1e-5
active_scheduler = True

In [ ]:
"""
Réseau de neurones avec PyTorch pour la régression:
"""

# Vérifier si CUDA est disponible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de: {device}")

# Créer un dataset PyTorch personnalisé
class CO2Dataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        
        if y is not None:
            self.y = torch.tensor(y.values, dtype=torch.float32).reshape(-1, 1)
            self.has_target = True
        else:
            self.has_target = False
            
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.has_target:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# Définition du modèle de réseau de neurones
class CO2RegressionNet(nn.Module):
    def __init__(self, input_dim):
        super(CO2RegressionNet, self).__init__()
        
        # Architecture plus stable
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            
            
            nn.Linear(64, 1)
        )
        
    def forward(self, x):
        return self.model(x)
    
    def __str__(self):
        string = "Description du network utilisé:\n"
        for i, layer in enumerate(self.model):
            string += f"Layer {i}: {layer}\n"
        return string
    


# Création des datasets et dataloaders
train_dataset = CO2Dataset(X_train_nn, y_train_nn)
val_dataset = CO2Dataset(X_val, y_val)
test_dataset = CO2Dataset(X_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)



Utilisation de: cpu


In [123]:

# Initialisation du modèle, fonction de perte et optimiseur
input_dim = X_train.shape[1]
model = CO2RegressionNet(input_dim)

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        m.bias.data.fill_(0.01)


try:
    # Chargement des poids sauvegardés
    nomModel = input("Prendre un modèle ? (entrez le nom du fichier en entier ou non pour continuer sans) : ")
    if nomModel != "non":
        print("Chargement du modèle", nomModel+".pth")
        model.load_state_dict(torch.load(nomModel + '.pth'))
        
    else:
        model.apply(init_weights)
        print("Vous avez décidé de ne pas charger de modèle. Initialisation d'un nouveau modèle")

except FileNotFoundError:
    model.apply(init_weights)
    print("Pas de modèle sauvegardé trouvé, initialisation d'un nouveau modèle.")


        
model.to(device)


criterion = nn.L1Loss()  # Mean Squared Error pour la régression
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',
    factor=0.85,       
    patience=3,
    min_lr=1e-6,
    threshold=0.01,     
)

Vous avez décidé de ne pas charger de modèle. Initialisation d'un nouveau modèle


In [124]:
# Fonction d'entraînement
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, scheduler=None):
    train_losses = []
    val_losses = []
    
    best_val_loss = float('inf')
    best_model_state = None
    
    for epoch in range(num_epochs):
        # Mode entraînement
        model.train()
        running_loss = 0.0
        
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            # Remise à zéro des gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            # Backward pass et optimisation
            loss.backward()
            
            # Clip gradients
            # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            running_loss += loss.item() * X_batch.size(0)
        
        epoch_train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_train_loss)
        
        # Mode évaluation
        model.eval()
        running_loss = 0.0
        
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                
                running_loss += loss.item() * X_batch.size(0)
        
        epoch_val_loss = running_loss / len(val_loader.dataset)
        val_losses.append(epoch_val_loss)
        
        # Ajustement du learning rate
        if scheduler is not None and active_scheduler:
            # Comme c'est un ReduceLROnPlateau, nous lui passons la perte de validation
            scheduler.step(epoch_val_loss)
        
        # Récupération du learning rate actuel pour l'affichage
        current_lr = optimizer.param_groups[0]['lr']
        
        # Affichage des métriques
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_train_loss:.6f}, Val Loss: {epoch_val_loss:.6f}, LR: {current_lr:.6f}')
        
        # Sauvegarde du meilleur modèle
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_model_state = model.state_dict().copy()
    
    # Chargement du meilleur modèle
    model.load_state_dict(best_model_state)
    
    return model, train_losses, val_losses

In [125]:
# Entraînement du modèle

print("Démarage de l'entraînement avec les paramètres suivants:")
print(f"-\t Learning Rate: {LR}")
print(f"-\t Weight Decay: {WEIGHT_DECAY}")
print(f"-\t Active Scheduler: {active_scheduler}")
print(f"-\t Batch Size: {BATCH_SIZE}")
print(f"-\t Raw CO2 - Min: {train_df['co2'].min()}, Max: {train_df['co2'].max()}, Mean: {co2_mean}, Std: {co2_std}")
print("")
print(f"Réseau de neurones: \n{model}")

print("\n")


trained_model, train_losses, val_losses = train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=NUM_EPOCHS, scheduler=scheduler
)

torch.save(trained_model.state_dict(), 'co2_regression_model.pth')
print("Modèle sauvegardé dans 'co2_regression_model.pth'")

# Graphique des pertes
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Courbes d\'apprentissage')
plt.legend()
plt.grid(True)
plt.savefig('nn_training_loss.png')
plt.close()

# Évaluation sur l'ensemble de validation
model.eval()
val_predictions = []
val_targets = []

with torch.no_grad():
    for X_batch, y_batch in val_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        val_predictions.extend(outputs.cpu().numpy())
        val_targets.extend(y_batch.numpy())

val_predictions = np.array(val_predictions).flatten()
val_predictions = val_predictions * co2_std + co2_mean
val_targets = np.array(val_targets).flatten()
val_targets = val_targets * co2_std + co2_mean

val_mae = mean_absolute_error(val_targets, val_predictions)
val_r2 = r2_score(val_targets, val_predictions)

print(f"Validation MAE: {val_mae:.4f}")
print(f"Validation R²: {val_r2:.4f}")











# Prédictions sur l'ensemble de test
model.eval()
test_predictions = []

with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        test_predictions.extend(outputs.cpu().numpy())

Y_test_nn = np.array(test_predictions).flatten()
Y_test_nn = Y_test_nn * co2_std + co2_mean

# Sauvegarde des résultats
resultat = [(test_df['id'].iloc[i], Y_test_nn[i]) for i in range(len(Y_test_nn))]
with open('resultat_pytorch.csv', 'w') as f:
    f.write("id,co2\n")
    for id, co2 in resultat:
        f.write(f"{id},{co2}\n")


Démarage de l'entraînement avec les paramètres suivants:
-	 Learning Rate: 1e-05
-	 Weight Decay: 1e-05
-	 Active Scheduler: True
-	 Batch Size: 64
-	 Raw CO2 - Min: 13, Max: 572, Mean: 201.6360859975277, Std: 33.9065497455555

Réseau de neurones: 
Description du network utilisé:
Layer 0: Linear(in_features=102, out_features=128, bias=True)
Layer 1: ReLU()
Layer 2: Linear(in_features=128, out_features=64, bias=True)
Layer 3: ReLU()
Layer 4: Linear(in_features=128, out_features=1, bias=True)





RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x64 and 128x1)